In [49]:
import pyzettle as pz
from dotenv import load_dotenv
import os
import pendulum
import pandas as pd

load_dotenv()
data = (
    pz.GetPayments(
        client_id=os.getenv("CLIENT_ID"),
        api_key=os.getenv("API_KEY"),
    )
    .fetch_purchases()
    .format_payments()
    .data
)
data


Payment columns ['amount', 'timestamp', 'purchaseNumber', 'products', 'discounts', 'customAmountSale', 'refunded', 'refund', 'gratuityAmount', 'type', 'attributes.acquirerMID', 'currency', 'attributes.acqSystemTraceAuditNr', 'attributes.mxCardType', 'attributes.mxFiid', 'referenceNumber', 'attributes.installmentAmount', 'attributes.nrOfInstallments', 'references.refundsPayment', 'receiverOrganization', 'references.checkoutUUID', 'details.checkoutUUID', 'attributes.cardIssuingBank']


,timestamp,purchase_number,discounts,custom_amount_sale,refunded,refund,payment_type,attributes.acquirerMID,attributes.acqSystemTraceAuditNr,quantity,unit_price,product
0,2024-09-28T21:47:56.938+0000,5756,[],False,False,False,card,61905246,NaN,1,360,Top-up: Custom
1,2024-09-28T21:41:45.988+0000,5755,[],False,False,False,card,61905246,NaN,1,1000,Top-up: £10
2,2024-09-28T21:26:40.809+0000,5754,[],False,False,False,card,61905246,NaN,1,1000,Top-up: £10
3,2024-09-28T21:24:14.995+0000,5753,[],False,False,False,cash,NaN,NaN,1,500,Top-up: Custom
4,2024-09-28T21:18:45.287+0000,5752,[],False,False,False,card,61905246,NaN,1,1000,Top-up: £10
...,...,...,...,...,...,...,...,...,...,...,...,...
6972,2022-09-20T13:35:06.730+0000,4,[],False,False,False,giftcard,NaN,NaN,1,1500,Entry Ticket
6973,2022-09-20T13:33:37.035+0000,3,[],False,False,False,cash,NaN,NaN,1,1500,Gift card
6974,2022-09-18T16:13:09.773+0000,2,[],False,False,False,cash,NaN,NaN,1,1500,Entry Ticket
6975,2022-09-18T16:13:09.773+0000,2,[],False,False,False,cash,NaN,NaN,1,1000,Top Up Ticket


In [78]:
def split_quantities(df: pd.DataFrame) -> pd.DataFrame:
    # Get absolute value of quantities
    df = df.assign(quantity=df["quantity"].astype(int).abs())

    # Use index.repeat to create multiple rows based on quantity
    result_df = df.loc[df.index.repeat(df["quantity"])].copy()

    # Set all quantities to 1
    result_df["quantity"] = 1

    # Reset index
    return result_df.reset_index(drop=True)


data = split_quantities(data)


In [80]:
def filter_data(data: pd.DataFrame, days: list[str], products: list[str] = None):
    data = data[data["timestamp"].dt.day_name().isin(days)]

    if products is not None:
        data = data[data["product"].isin(products)]  # Use the correct column name

    return data

# Usage
data = filter_data(data, days=["Friday"], products=["Samosa"])


In [82]:
def resample_data(df: pd.DataFrame):
    # Set timestamp as index
    df.set_index("timestamp", inplace=True)

    # Resample by hour, summing 'quantity', averaging 'unit_price', and keeping first non-null values for other columns
    return df.resample("H").agg(
        {
            "quantity": "sum",
            "unit_price": "sum",
            "product": "first",
            "payment_type": "first",
            # ... other columns with appropriate aggregations
        }
    )


resample_data(data)


/var/folders/wk/dw6zmc916ql2b6f3bjd1ynn9k9qhvy/T/ipykernel_27620/81195788.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  return df.resample("H").agg(


,quantity,unit_price,product,payment_type
timestamp,,,,
2024-09-27 16:00:00+00:00,1,100,Samosa,card
2024-09-27 17:00:00+00:00,12,1200,Samosa,card
2024-09-27 18:00:00+00:00,13,1300,Samosa,card
2024-09-27 19:00:00+00:00,5,500,Samosa,card
2024-09-27 20:00:00+00:00,5,500,Samosa,card
2024-09-27 21:00:00+00:00,16,1600,Samosa,card
